In [ ]:
# |default_exp error_correction_t5

In [ ]:
# | export

def preprocess_function(examples, tokenizer):
    model_inputs = tokenizer(examples["ocr"])

    labels = tokenizer(text_target=examples["gs"])

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()